<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/Google_Earth_Engine_Maxent_%EA%B5%AC%ED%98%84_%EA%B8%B0%EB%8A%A5_%EC%86%8C%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [2]:
# Create some sample species presence/absence training data.

# 출현/부재 훈련 데이터 생성
training_data = ee.FeatureCollection([
    # 종 출현 지점
    ee.Feature(ee.Geometry.Point([-122.39567, 38.02740]), {'presence': 1}),
    ee.Feature(ee.Geometry.Point([-122.68560, 37.83690]), {'presence': 1}),
    # 종 부재 지점
    ee.Feature(ee.Geometry.Point([-122.59755, 37.92402]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.47137, 37.99291]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.52905, 37.85642]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.03010, 37.66660]), {'presence': 0})
])

In [3]:
# Landsat 8 이미지를 가져와 반사율 밴드 선택
image = (ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_044034_20200606')
         .select(['SR_B[1-7]'])
         .multiply(0.0000275).add(-0.2))  # 축척 계수 적용

In [4]:
# 종 출현/부재 위치에서 이미지 샘플링
training = image.sampleRegions(**{
    'collection': training_data,
    'scale': 30
})

In [5]:
# 이미지 샘플링된 지점에서 Maxent 분류기 정의 및 훈련
classifier = ee.Classifier.amnhMaxent().train(**{
    'features': training,
    'classProperty': 'presence',
    'inputProperties': image.bandNames()
})

In [6]:
# Maxent 분류기를 사용하여 이미지 분류
image_classified = image.classify(classifier)

In [7]:
# 지도 생성
m = geemap.Map()

# 분류된 이미지를 지도에 추가
m.addLayer(image_classified, {
    'bands': ["probability"],
    'max': 0.8,
    'min': 0.09,
    'opacity': 1,
    'palette': ["d8b365", "f5f5f5", "5ab4ac"]
}, 'Suitability Map')

# 지도 표시
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…